In [ ]:
! pip install langchain
! pip install langchain_openai
! pip install langchain_community
! pip install pymongo
! pip install python-dotenv
! pip install azure-core
! pip install azure-cosmos

In [ ]:
import json
import openai
import pymongo

from dotenv import dotenv_values
from openai import AzureOpenAI

env_name = "example.env" 
config = dotenv_values(env_name)

openai.api_type = config['openai_type']
openai.api_key = config['openai_api_key']
openai.api_base = config['openai_api_endpoint']
openai.api_version = config['openai_api_version']

client = AzureOpenAI(
    api_key=openai.api_key,
    api_version=openai.api_version,
    azure_endpoint = openai.api_base
)

In [16]:
# Connection string
mongo_conn = config['mongo_vcore_connection_string']
mongo_client = pymongo.MongoClient(mongo_conn)

# Database name
DATABASE_NAME = "ResturantChain"
db = mongo_client[DATABASE_NAME]

# Drop the database if it already exists (consider if this is really needed, as it will remove all existing data)
#mongo_client.drop_database(DATABASE_NAME)

# Collection names
collection_names = ["Contoso", "Contoso West", "Users"]

# Iterate through the collection names and create them if they do not exist
for collection_name in collection_names:
    if collection_name not in db.list_collection_names():
        # Creates a collection
        db.create_collection(collection_name)
        print(f"Created collection '{collection_name}'.")
    else:
        print(f"Using existing collection: '{collection_name}'.")

ConfigurationError: The resolution lifetime expired after 20.015 seconds: Server Do53:10.50.10.50@53 answered The DNS operation timed out.; Server Do53:10.50.50.50@53 answered The DNS operation timed out.; Server Do53:75.75.75.75@53 answered The DNS operation timed out.; Server Do53:75.75.76.76@53 answered The DNS operation timed out.; Server Do53:10.50.10.50@53 answered The DNS operation timed out.; Server Do53:10.50.50.50@53 answered The DNS operation timed out.; Server Do53:75.75.75.75@53 answered The DNS operation timed out.; Server Do53:75.75.76.76@53 answered The DNS operation timed out.; Server Do53:10.50.10.50@53 answered The DNS operation timed out.; Server Do53:10.50.50.50@53 answered The DNS operation timed out.

In [ ]:
db.command({
  'createIndexes': 'FoodCollection',
  'indexes': [
    {
      'name': 'vectorSearchIndex',
      'key': {
        "Embedding": "cosmosSearch"
      },
      'cosmosSearchOptions': {
        'kind': 'vector-hnsw',
        'm': 16,
        'efConstruction': 40,
        'similarity': 'COS',
        'dimensions': 1536
      }
    }
  ]
});

In [ ]:
data_file = open(file="./data/output_data.json", mode="r") 
data = json.load(data_file)
data_file.close()
collection = db['Contoso']
result = collection.insert_many(data)

print(f"Number of data points added: {len(result.inserted_ids)} in {collection.name}")